In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import glob
import os

In [2]:
data = {}

for path in glob.glob(os.path.expanduser("~/results/*.json")):
  for typename, items in json.load(open(path)).items():
    if typename in data:
      data[typename].update(items)
    else:
      data[typename] = items

frames = { typename: pd.DataFrame.from_dict(items, orient='index') for typename, items in data.items() }

In [15]:
frames['program_run'][frames['algorithm_run'].groupby('program_run_id').size() < 42]['graph']


470fe0c6-04df-41d0-8e25-b3870debcec8    ../../../data/uk-2002.metis.graph
6d57ebf9-04c1-414b-874c-8d4395065410    ../../../data/uk-2002.metis.graph
b067797e-5170-475c-a334-6c616ef84fba    ../../../data/uk-2002.metis.graph
b89c9c40-8726-4289-80e8-4aede2eed85e    ../../../data/uk-2002.metis.graph
Name: graph, dtype: object

In [19]:
frames['algorithm_level'] \
    .loc[lambda x: x.level == 0] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'].loc[lambda x: x.graph.str.contains("in-")], left_on='program_run_id', right_index=True) \
    .loc[lambda x: x.algorithm.str.contains("sequential")] \
    ['cluster_count'] \
    .mean()

85513.875

In [45]:
frames['algorithm_level'] \
    .loc[lambda x: x.level == 0] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'].loc[lambda x: x.graph.str.contains("in-")], left_on='program_run_id', right_index=True) \
    .merge(frames['partition'], left_on='partition_id', right_index=True) \
    .loc[lambda x: x.algorithm_x.str.contains("partitioned")] \
    .groupby(['algorithm_y', 'size', 'allow_move_to_ghosts'])['cluster_count'].mean()

algorithm_y                                            size  allow_move_to_ghosts
chunk                                                  4     False                     90117
                                                             True                      91540
                                                       32    False                     96746
                                                             True                      95151
                                                       128   False                    107418
                                                             True                     100265
                                                       1024  False                    198608
                                                             True                     151895
cluster_based                                          4     False                     86100
                                                             True                

In [48]:
graphs = frames['program_run'].loc[lambda x: x.graph.str.contains("in-")]
algos = frames['algorithm_run'].merge(graphs, left_on='program_run_id', right_index=True)
frames['partition_element'] \
    .merge(frames['partition'], left_on='partition_id', right_index=True) \
    .merge(algos, left_on='partition_id', right_on='partition_id') \
    .groupby(['size', 'algorithm_x'])['nodes_in_ghost_clusters'].mean()

size  algorithm_x                                          
4     chunk                                                      2464.375000
      cluster_based                                               424.937500
      deterministic_greedy_with_linear_penalty                  45805.937500
      random                                                   273058.437500
      random_order_deterministic_greedy_with_linear_penalty     86861.312500
32    chunk                                                       550.179688
      cluster_based                                               211.562500
      deterministic_greedy_with_linear_penalty                   7916.148438
      random                                                    42184.695312
      random_order_deterministic_greedy_with_linear_penalty     14536.632812
128   chunk                                                       249.732422
      cluster_based                                               232.957031
      determinis

In [63]:
graphs = frames['program_run'].loc[lambda x: x.graph.str.contains("in-")]
algos = frames['algorithm_run'].merge(graphs, left_on='program_run_id', right_index=True)
frames['partition_element'] \
    .merge(frames['partition'], left_on='partition_id', right_index=True) \
    .merge(algos, left_on='partition_id', right_on='partition_id') \
    [['size', 'algorithm_x', 'connected_components']][:1]['connected_components']

0    [3, 1, 11, 1, 3, 2, 2, 1, 3, 2, 2, 1, 1, 1, 1,...
Name: connected_components, dtype: object